In [2]:
import sys
import pandas as pd
import io
import os
import ast
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
disease_samples = ['SRR12043176', 'SRR12043179','SRR12043177']
control_samples = ['SRR7721287', 'SRR12043175', 'SRR12043176']

In [5]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

In [18]:
main_df = []
common = dict()
main_directory = '/data/users-workspace/ruchika.sharma/tmp_results/variants'
for i,file in tqdm(enumerate(os.listdir(main_directory))):
    if i == 6:
        break
    if not file.endswith("_finalSNP.vcf"):
        continue
    print(file)
    df = read_vcf(os.path.join(main_directory, file))
    df['Sample_id'] = os.path.basename(file.split('_')[0])
    df['variant_details'] = df['INFO'].str.split(';').str[-1]
    df['Gene'] = df['variant_details'].str.split('|').str[3]
    df['RS_ID'] = df['INFO'].str.split('|').str[17]
    del df['INFO']
    del df['variant_details']
    del df['QUAL']
    df = df.loc[:,~df.columns.str.startswith('SRR')]
    df.drop(df.iloc[:, 4:7], inplace=True, axis=1)
    control_df = df.loc[df['Sample_id'].isin(control_samples)]
    disease_df = df.loc[df['Sample_id'].isin(disease_samples)]
    del control_df['Sample_id']
    del disease_df['Sample_id']
    control_df.insert(0, 'Type', 'Control')
    disease_df.insert(0, 'Type', 'Disease')
    control_df = control_df.set_index('Type')
    disease_df = disease_df.set_index('Type')
    combine_df = pd.merge(disease_df, control_df, how='outer', indicator=True)
    print(combine_df)
    #     combine_df = pd.concat([control_df, disease_df])
    #     combine_df.drop_duplicates(keep=False)
    #     main_df.append(combine_df)

2it [00:00, 12.50it/s]

SRR12043174_finalSNP.vcf
Empty DataFrame
Columns: [CHROM, POS, ID, REF, Gene, RS_ID, _merge]
Index: []
SRR12043175_finalSNP.vcf
                CHROM        POS ID REF           Gene       RS_ID      _merge
0                chr1    9323103  .   G          SPSB1              right_only
1                chr1   90732249  .   G                             right_only
2                chr1   93907096  .   G           GCLM              right_only
3                chr1  100156458  .   A         LRRC39              right_only
4                chr1  107570832  .   T           VAV3              right_only
5                chr1  108573531  .   C        FAM102B              right_only
6                chr1  110718857  .   A                             right_only
7                chr1  117702494  .   C   RP11-134N8.2              right_only
8                chr1  181079515  .   A                             right_only
9                chr1  181079517  .   A                             right_only
10 

6it [00:00, 13.96it/s]

     CHROM        POS ID REF           Gene       RS_ID _merge
0     chr1   90732249  .   G                              both
1     chr1   93165365  .   G          TMED5               both
2     chr1  107570649  .   A           VAV3  rs79910174   both
3     chr1  107570650  .   T           VAV3  rs77203866   both
4     chr1  107570651  .   T           VAV3  rs78622330   both
5     chr1  107570832  .   T           VAV3               both
6     chr1  108573531  .   C        FAM102B               both
7     chr1  110718857  .   A                              both
8     chr1  177864073  .   A                              both
9     chr1  196608152  .   C          KCNT2               both
10    chr1  206663509  .   C          DYRK3               both
11    chr1  225645848  .   A           ENAH               both
12   chr10    7298457  .   G         SFMBT2               both
13   chr10    7298458  .   G         SFMBT2               both
14   chr10    7298461  .   T         SFMBT2            

In [14]:
main_df

[                CHROM        POS ID REF    Sample_id           Gene  \
 0                chr1    9323103  .   G  SRR12043174          SPSB1   
 1                chr1   90732249  .   G  SRR12043174                  
 2                chr1   93907096  .   G  SRR12043174           GCLM   
 3                chr1  100156458  .   A  SRR12043174         LRRC39   
 4                chr1  101774940  .   G  SRR12043174      LINC01709   
 5                chr1  101774944  .   A  SRR12043174      LINC01709   
 6                chr1  107570832  .   T  SRR12043174           VAV3   
 7                chr1  108573531  .   C  SRR12043174        FAM102B   
 8                chr1  110718857  .   A  SRR12043174                  
 9                chr1  117702494  .   C  SRR12043174   RP11-134N8.2   
 10               chr1  181079515  .   A  SRR12043174                  
 11               chr1  181079517  .   A  SRR12043174                  
 12               chr1  196608152  .   C  SRR12043174          K

In [3]:
#####2. Snps.tab variants processing

# main_df = []
# common = dict()
# main_directory = '/data/users-workspace/ruchika.sharma/tmp_results/variants'
# for i,file in tqdm(enumerate(os.listdir(main_directory))):
#     if not file.endswith("_finalSNP.vcf"):
#         continue
#     fullPath = os.path.join(main_directory, file)
#     for j, var_files in enumerate(os.listdir(fullPath)):
#         if not var_files.endswith("snps.tab"):
#             continue
#         df = pd.read_csv(os.path.join(fullPath, var_files), sep = '\t')
#         df = df[df['LOCUS_TAG'].notnull()]
#         df['Sample_id'] = os.path.basename(file)
#         df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]], axis=1)
#         main_df.append(df)

43it [00:05,  7.94it/s]


In [4]:
bacterial_mutations = pd.concat(main_df)

In [5]:
bacterial_mutations

,EFFECT,LOCUS_TAG,GENE,PRODUCT,Sample_id
1,synonymous_variant c.57T>G p.Arg19Arg,b0002,thrA,fused aspartate kinase/homoserine dehydrogenase 1,1768195
2,synonymous_variant c.223T>C p.Leu75Leu,b0002,thrA,fused aspartate kinase/homoserine dehydrogenase 1,1768195
3,missense_variant c.235G>A p.Ala79Thr,b0002,thrA,fused aspartate kinase/homoserine dehydrogenase 1,1768195
4,synonymous_variant c.252G>A p.Gly84Gly,b0002,thrA,fused aspartate kinase/homoserine dehydrogenase 1,1768195
5,synonymous_variant c.268T>C p.Leu90Leu,b0002,thrA,fused aspartate kinase/homoserine dehydrogenase 1,1768195
6,synonymous_variant c.285T>C p.Asp95Asp,b0002,thrA,fused aspartate kinase/homoserine dehydrogenase 1,1768195
7,synonymous_variant c.411A>G p.Val137Val,b0002,thrA,fused aspartate kinase/homoserine dehydrogenase 1,1768195
8,synonymous_variant c.423C>T p.Arg141Arg,b0002,thrA,fused aspartate kinase/homoserine dehydrogenase 1,1768195
9,synonymous_variant c.438T>C p.Thr146Thr,b0002,thrA,fused aspartate kinase/homoserine dehydrogenase 1,1768195
10,synonymous_variant c.462G>A p.Leu154Leu,b0002,thrA,fused aspartate kinase/homoserine dehydrogenase 1,1768195


In [6]:
del bacterial_mutations['PRODUCT']

In [7]:
bacterial_mutations

,EFFECT,LOCUS_TAG,GENE,Sample_id
1,synonymous_variant c.57T>G p.Arg19Arg,b0002,thrA,1768195
2,synonymous_variant c.223T>C p.Leu75Leu,b0002,thrA,1768195
3,missense_variant c.235G>A p.Ala79Thr,b0002,thrA,1768195
4,synonymous_variant c.252G>A p.Gly84Gly,b0002,thrA,1768195
5,synonymous_variant c.268T>C p.Leu90Leu,b0002,thrA,1768195
6,synonymous_variant c.285T>C p.Asp95Asp,b0002,thrA,1768195
7,synonymous_variant c.411A>G p.Val137Val,b0002,thrA,1768195
8,synonymous_variant c.423C>T p.Arg141Arg,b0002,thrA,1768195
9,synonymous_variant c.438T>C p.Thr146Thr,b0002,thrA,1768195
10,synonymous_variant c.462G>A p.Leu154Leu,b0002,thrA,1768195


In [8]:
bacterial_mutations = bacterial_mutations[bacterial_mutations['EFFECT'].notna()]

In [9]:
bacterial_mutations[['VARIANT TYPE','NUCLEOTIDE', 'PROTEIN']] = bacterial_mutations.EFFECT.str.split(expand=True) 

In [10]:
del bacterial_mutations['EFFECT']

In [11]:
bacterial_mutations['GENE'] = bacterial_mutations['GENE'].str.lower()

In [12]:
bacterial_mutations['GENE'].value_counts()

yehi    2281
entf    1989
hrpa    1924
yghj    1915
gltb    1913
yfhm    1868
yfas    1813
meth    1740
purl    1712
mdtc    1703
muts    1602
puta    1586
yeej    1566
carb    1553
mukb    1546
recb    1545
yphg    1498
yfal    1459
uvra    1458
yehm    1439
narz    1417
mdtb    1400
msck    1367
dgce    1352
znta    1350
evgs    1343
iles    1335
bcsc    1325
mscm    1287
frya    1265
        ... 
pmrr       1
aspu       1
yfes       1
leul       1
chix       1
ylbi       1
chbb       1
mcrb       1
rmf        1
yhby       1
rpme       1
pyrl       1
rpsq       1
arcz       1
ymdg       1
omrb       1
yidd       1
rpmg       1
yjiv       1
alaw       1
lysy       1
wbbh       1
rrfc       1
cyar       1
ryjb       1
ydfv       1
cspe       1
bola       1
yfjq       1
argw       1
Name: GENE, Length: 4243, dtype: int64

In [33]:
bacterial_mutations[bacterial_mutations['GENE'].isin(['lptd','bama','tolc','fepa','fadl','ompa','tama'])].to_csv("ecoli_mutation_without_groupby.csv")

In [13]:
bacterial_mutations

,LOCUS_TAG,GENE,Sample_id,VARIANT TYPE,NUCLEOTIDE,PROTEIN
1,b0002,thra,1768195,synonymous_variant,c.57T>G,p.Arg19Arg
2,b0002,thra,1768195,synonymous_variant,c.223T>C,p.Leu75Leu
3,b0002,thra,1768195,missense_variant,c.235G>A,p.Ala79Thr
4,b0002,thra,1768195,synonymous_variant,c.252G>A,p.Gly84Gly
5,b0002,thra,1768195,synonymous_variant,c.268T>C,p.Leu90Leu
6,b0002,thra,1768195,synonymous_variant,c.285T>C,p.Asp95Asp
7,b0002,thra,1768195,synonymous_variant,c.411A>G,p.Val137Val
8,b0002,thra,1768195,synonymous_variant,c.423C>T,p.Arg141Arg
9,b0002,thra,1768195,synonymous_variant,c.438T>C,p.Thr146Thr
10,b0002,thra,1768195,synonymous_variant,c.462G>A,p.Leu154Leu


In [35]:
variant_type = bacterial_mutations['VARIANT TYPE'].value_counts()
variant_type

synonymous_variant                                 18994
missense_variant                                     808
missense_variant&conservative_inframe_insertion       42
missense_variant&conservative_inframe_deletion        38
missense_variant&disruptive_inframe_deletion          32
disruptive_inframe_deletion                           10
frameshift_variant&missense_variant                    3
frameshift_variant&synonymous_variant                  1
Name: VARIANT TYPE, dtype: int64

In [36]:
bacterial_mutation_burden = bacterial_mutations.groupby(["PROTEIN", "GENE"]).size()

In [37]:
bacterial_mutation_burden

PROTEIN                                    GENE
p.144                                      oprn     44
p.152                                      oprn     87
p.154                                      oprn      1
p.186                                      oprn     26
p.190                                      oprn     26
p.193                                      oprn     24
p.195                                      oprn      1
p.199                                      bama      1
p.287                                      oprn      8
p.332                                      oprn      5
p.340                                      oprn     66
p.389                                      bama     24
p.442                                      bama     23
p.577                                      bama     68
p.607                                      bama    488
p.662                                      bama     91
p.686                                      bama     47
p.72             

In [19]:
missense_variants = bacterial_mutations.loc[bacterial_mutations['VARIANT TYPE'].str.contains('missense_variant')]

In [28]:
missense_variants = missense_variants[missense_variants['GENE'].isin(['lptd','bama','tolc','fepa','fadl','ompa','tama'])]

In [15]:
missense_variants

,LOCUS_TAG,GENE,Sample_id,VARIANT TYPE,NUCLEOTIDE,PROTEIN
3,b0002,thra,1768195,missense_variant,c.235G>A,p.Ala79Thr
26,b0002,thra,1768195,missense_variant,c.1772A>T,p.Gln591Leu
27,b0002,thra,1768195,missense_variant,c.1780T>C,p.Tyr594His
36,b0002,thra,1768195,missense_variant,c.2259_2261delCGTinsTGC,p.Val754Ala
50,b0003,thrb,1768195,missense_variant,c.822A>T,p.Glu274Asp
57,b0004,thrc,1768195,missense_variant,c.708_709delGAinsAG,p.Thr237Ala
67,b0005,yaax,1768195,missense_variant,c.209T>C,p.Leu70Pro
69,b0005,yaax,1768195,missense_variant,c.251C>A,p.Pro84His
74,b0006,yaaa,1768195,missense_variant,c.221C>A,p.Ala74Glu
79,b0007,yaaj,1768195,missense_variant,c.971G>A,p.Gly324Asp


In [20]:
missense_variants = missense_variants.groupby(["PROTEIN", "GENE"]).size().reset_index(name="Counts")

In [23]:
missense_variants.sort_values(by="Counts", ascending=False)

,PROTEIN,GENE,Counts
37097,p.Val230Gly,oppd,20
14007,p.Gln5Pro,sgbu,20
2894,p.Ala319Asp,msrp,20
18986,p.His148Arg,mpl,20
3601,p.Ala422Ser,ligb,20
37550,p.Val29Ala,tory,20
29186,p.Ser12Thr,cred,20
20624,p.Ile216Thr,rph,20
13408,p.Gln293Leu,pros,20
7991,p.Asn143Ser,caie,20


In [26]:
missense_variants.to_csv("Ecoli_complete_list_of_mutations.csv")

In [35]:
missense_variants.to_csv("ecoli_mutation_groupby.csv")

In [51]:
missense_variants.reset_index()

,PROTEIN,GENE,0
0,p.Ala100Thr,oprn,1
1,p.Ala113Ser,oprn,1
2,p.Ala12Thr,oprn,2
3,p.Ala140Thr,oprn,1
4,p.Ala365Val,oprn,1
5,p.Ala371Gly,oprn,7
6,p.Ala38Thr,oprn,2
7,p.Ala410Ser,oprn,38
8,p.Ala4Thr,oprn,31
9,p.Ala667fs,bama,1


In [23]:
bacterial_mutations_final.to_csv("Combined_mutations_filtered_and_separated_removedNAN_matrix.txt")